# kramer2.0 Experiment 02: Analyses
## Angelique I. Delarazan
### kramer2.0_experiment_02_vA5
### Retrieval Tasks: Character-Cued Recall and Timeline Task

## Set Up

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from scipy import stats
import itertools
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pingouin as pg
from pingouin import anova as pg_anova
from statannot import add_stat_annotation

## Charcter-Cued Recall

In [3]:
recall = pd.read_csv('~/Box Sync/aidelarazan_box/Projects/kramer2.0/data/kramer2.0_vA5/kramer2.0_vA5_sub-all_desc-recall.csv')
recall['condition'] = recall['coherence'] + '_' + recall['lag']
print(recall.columns.tolist())
recall.head()

['subject', 'version', 'character', 'coherence', 'lag', 'plot', 'response', 'rt', 'stimulus', 'word_count', 'condition']


,subject,version,character,coherence,lag,plot,response,rt,stimulus,word_count,condition
0,401,3,Sandra,unrelated,short,sideplot,"{""Q0"":""Sandra was going on a blind date with a...",183742.3,NaN,87,unrelated_short
1,401,3,Beatrice,coherent,long,sideplot,"{""Q0"":""Beatrice was having trouble with her gi...",180485.3,NaN,123,coherent_long
2,401,3,Melvin,unrelated,long,sideplot,"{""Q0"":""Melvin Doyle is a friend of the main ch...",180831.0,NaN,93,unrelated_long
3,401,3,Johnny,coherent,short,sideplot,"{""Q0"":""Johnny Pratt is a friend of the main ch...",180413.7,NaN,78,coherent_short
4,401,3,Charles,mainplot,mainplot,mainplot,"{""Q0"":""Charles Bort is the main character of t...",365204.1,NaN,354,mainplot_mainplot


In [5]:
aov = pg.rm_anova(
    data=recall[recall['coherence']!='mainplot'],
    dv='word_count',
    within=['coherence', 'lag'],
    subject='subject',
    detailed=True
)

pg.print_table(aov)

post_hoc = pg.pairwise_tests(
    data=recall[recall['coherence']!='mainplot'],
    dv='word_count',
    within='condition',
    subject='subject',
    padjust='bonf'
)

pg.print_table(post_hoc)


ANOVA SUMMARY

Source                  SS    ddof1    ddof2         MS       F    p-unc    p-GG-corr    ng2    eps
---------------  ---------  -------  -------  ---------  ------  -------  -----------  -----  -----
coherence        14200.694        1       35  14200.694  16.902    0.000        0.000  0.046  1.000
lag               1308.028        1       35   1308.028   0.993    0.326        0.326  0.004  1.000
coherence * lag    802.778        1       35    802.778   0.919    0.344        0.344  0.003  1.000


POST HOC TESTS

Contrast    A               B                Paired    Parametric        T     dof  alternative      p-unc    p-corr  p-adjust       BF10    hedges
----------  --------------  ---------------  --------  ------------  -----  ------  -------------  -------  --------  ----------  -------  --------
condition   coherent_long   coherent_short   True      True          1.339  35.000  two-sided        0.189     1.000  bonf          0.407     0.209
condition   coherent_l

In [9]:
ttest = pg.ttest(
    x=recall[recall['coherence']=='coherent'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    y=recall[recall['coherence']=='unrelated'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    paired=False
)

pg.print_table(ttest)

print()

    T    dof  alternative      p-val  CI95%            cohen-d    BF10    power
-----  -----  -------------  -------  -------------  ---------  ------  -------
2.597    142  two-sided        0.010  [ 4.74 34.98]      0.433   3.765    0.732




In [10]:
ttest = pg.ttest(
    x=recall[recall['lag']=='short'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    y=recall[recall['lag']=='long'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    paired=False
)

pg.print_table(ttest)

print()

     T    dof  alternative      p-val  CI95%              cohen-d    BF10    power
------  -----  -------------  -------  ---------------  ---------  ------  -------
-0.772    142  two-sided        0.442  [-21.47   9.42]      0.129   0.235    0.120




## Timeline Task

In [54]:
timeline = pd.read_csv('~/Box Sync/aidelarazan_box/Projects/kramer2.0/data/kramer2.0_vA5/kramer2.0_vA5_sub-all_desc-timeline.csv')
timeline['condition'] = timeline['coherence'] + '_' + timeline['lag']
timeline = timeline[timeline['exclusion']=='include']
print(timeline.columns.tolist())
timeline.head()

['subject', 'version', 'character', 'act', 'event', 'position', 'coherence', 'lag', 'plot', 'response', 'rt', 'error', 'abs_error', 'condition', 'exclusion']


,subject,version,character,act,event,position,coherence,lag,plot,response,rt,error,abs_error,condition,exclusion
2,401,3,Charles,1,A,1,mainplot,mainplot,mainplot,73.0,7473.4,72.0,72.0,mainplot_mainplot,include
5,401,3,Charles,1,D,52,mainplot,mainplot,mainplot,62.0,2287.2,10.0,10.0,mainplot_mainplot,include
6,401,3,Johnny,1,A,58,coherent,short,sideplot,94.0,4923.2,36.0,36.0,coherent_short,include
7,401,3,Beatrice,2,A,120,coherent,long,sideplot,123.0,2907.9,3.0,3.0,coherent_long,include
8,401,3,Charles,1,A,2,mainplot,mainplot,mainplot,64.0,3052.3,62.0,62.0,mainplot_mainplot,include


In [55]:
timeline = timeline[timeline['coherence']!='mainplot'].groupby(['subject', 'coherence', 'lag', 'condition']).mean().reset_index()

In [56]:
aov = pg.rm_anova(
    data=timeline[timeline['coherence']!='mainplot'],
    dv='error',
    within=['coherence', 'lag'],
    subject='subject',
    detailed=True
)

pg.print_table(aov)

post_hoc = pg.pairwise_tests(
    data=timeline[timeline['coherence']!='mainplot'],
    dv='error',
    within='condition',
    subject='subject',
    padjust='bonf'
)

pg.print_table(post_hoc)


ANOVA SUMMARY

Source                 SS    ddof1    ddof2        MS       F    p-unc    p-GG-corr    ng2    eps
---------------  --------  -------  -------  --------  ------  -------  -----------  -----  -----
coherence         200.347        1       35   200.347   1.000    0.324        0.324  0.005  1.000
lag              3039.613        1       35  3039.613  14.358    0.001        0.001  0.074  1.000
coherence * lag    59.274        1       35    59.274   0.181    0.673        0.673  0.002  1.000


POST HOC TESTS

Contrast    A               B                Paired    Parametric         T     dof  alternative      p-unc    p-corr  p-adjust      BF10    hedges
----------  --------------  ---------------  --------  ------------  ------  ------  -------------  -------  --------  ----------  ------  --------
condition   coherent_long   coherent_short   True      True          -2.945  35.000  two-sided        0.006     0.034  bonf         6.846    -0.692
condition   coherent_long   unre

In [57]:
aov = pg.rm_anova(
    data=timeline[timeline['coherence']!='mainplot'],
    dv='abs_error',
    within=['coherence', 'lag'],
    subject='subject',
    detailed=True
)

pg.print_table(aov)

post_hoc = pg.pairwise_tests(
    data=timeline[timeline['coherence']!='mainplot'],
    dv='abs_error',
    within='condition',
    subject='subject',
    padjust='bonf'
)

pg.print_table(post_hoc)


ANOVA SUMMARY

Source                 SS    ddof1    ddof2        MS       F    p-unc    p-GG-corr    ng2    eps
---------------  --------  -------  -------  --------  ------  -------  -----------  -----  -----
coherence          26.174        1       35    26.174   0.231    0.634        0.634  0.001  1.000
lag              2052.742        1       35  2052.742  15.539    0.000        0.000  0.089  1.000
coherence * lag   191.404        1       35   191.404   1.683    0.203        0.203  0.009  1.000


POST HOC TESTS

Contrast    A               B                Paired    Parametric         T     dof  alternative      p-unc    p-corr  p-adjust      BF10    hedges
----------  --------------  ---------------  --------  ------------  ------  ------  -------------  -------  --------  ----------  ------  --------
condition   coherent_long   coherent_short   True      True           2.045  35.000  two-sided        0.048     0.290  bonf         1.144     0.425
condition   coherent_long   unre

In [58]:
ttest = pg.ttest(
    x=timeline[timeline['coherence']=='coherent']['abs_error'],
    y=timeline[timeline['coherence']=='unrelated']['abs_error'],
    paired=True
)

pg.print_table(ttest)

print()

     T    dof  alternative      p-val  CI95%            cohen-d    BF10    power
------  -----  -------------  -------  -------------  ---------  ------  -------
-0.478     71  two-sided        0.634  [-4.41  2.71]      0.067   0.145    0.086




In [59]:
ttest = pg.ttest(
    x=timeline[timeline['lag']=='short']['abs_error'],
    y=timeline[timeline['lag']=='long']['abs_error'],
    paired=True
)

pg.print_table(ttest)

print()

     T    dof  alternative      p-val  CI95%              cohen-d     BF10    power
------  -----  -------------  -------  ---------------  ---------  -------  -------
-4.071     71  two-sided        0.000  [-11.25  -3.85]      0.618  173.665    0.999




In [64]:
ttest = pg.ttest(
    x=timeline[timeline['coherence']=='coherent']['error'],
    y=timeline[timeline['coherence']=='unrelated']['error'],
    paired=True
)

pg.print_table(ttest)

print()

    T    dof  alternative      p-val  CI95%            cohen-d    BF10    power
-----  -----  -------------  -------  -------------  ---------  ------  -------
0.876     71  two-sided        0.384  [-3.01  7.73]      0.138   0.187    0.212




In [65]:
ttest = pg.ttest(
    x=timeline[timeline['lag']=='short']['error'],
    y=timeline[timeline['lag']=='long']['error'],
    paired=True
)

pg.print_table(ttest)

print()

    T    dof  alternative      p-val  CI95%            cohen-d    BF10    power
-----  -----  -------------  -------  -------------  ---------  ------  -------
3.375     71  two-sided        0.001  [ 3.76 14.62]      0.558  21.344    0.997


